In [1]:
from __future__ import absolute_import, division, print_function

import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow.keras import layers

tf.VERSION

W0512 01:05:03.032977 139838094464832 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


'1.12.0'